In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.initializers import glorot_normal, normal
from deepreplay.callbacks import ReplayData
from deepreplay.replay import Replay
from deepreplay.plot import compose_plots
import keras.initializers
from matplotlib import pyplot as plt
import keras

## 1.
Explain vanishing gradients phenomenon using standard normalization with different values of standard deviation and tanh and sigmoid activation functions. Then show how Xavier (aka Glorot normal) initialization of weights helps in dealing with this problem. Next use ReLU activation and show that instead of Xavier initialization, He initialization works better for ReLU activation. You can plot activations at each of the 5 layers to answer this question.

In [ ]:
def build_model(n_layers, input_dim, units, activation, initializer):
    if isinstance(units, list):
        assert len(units) == n_layers
    else:
        units = [units] * n_layers
        
    model = Sequential()
    # Adds first hidden layer with input_dim parameter
    model.add(Dense(units=units[0], 
                    input_dim=input_dim, 
                    activation=activation,
                    kernel_initializer=initializer, 
                    name='h1'))
    
    # Adds remaining hidden layers
    for i in range(2, n_layers + 1):
        model.add(Dense(units=units[i-1], 
                        activation=activation, 
                        kernel_initializer=initializer, 
                        name='h{}'.format(i)))
    
    # Adds output layer
    model.add(Dense(units=1, activation='sigmoid', kernel_initializer=initializer, name='o'))
    # Compiles the model
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['acc'])
    return model

In [ ]:
def plot_initializations(filename, group_name, initializer, activation, output_root):
        model = build_model(n_layers=5, input_dim=10, units=100, 
                    activation=activation, initializer=initializer)
        # Since we only need initial weights, we don't even need to train the model! 
        # We still use the ReplayData callback, but we can pass the model as argument instead
        h5_file = f'{filename}.h5'
        replaydata = ReplayData(X, y, filename=h5_file, group_name=group_name, model=model)

        # Now we feed the data to the actual Replay object
        # so we can build the visualizations
        replay = Replay(replay_filename=h5_file, group_name=group_name)

        # Using subplot2grid to assemble a complex figure...
        fig = plt.figure(figsize=(12, 6))
        ax_zvalues = plt.subplot2grid((2, 2), (0, 0))
        ax_weights = plt.subplot2grid((2, 2), (0, 1))
        ax_activations = plt.subplot2grid((2, 2), (1, 0))
        ax_gradients = plt.subplot2grid((2, 2), (1, 1))

        wv = replay.build_weights(ax_weights)
        gv = replay.build_gradients(ax_gradients)
        # Z-values
        zv = replay.build_outputs(ax_zvalues, before_activation=True, 
                                exclude_outputs=True, include_inputs=False)
        # Activations
        av = replay.build_outputs(ax_activations, exclude_outputs=True, include_inputs=False)

        # Finally, we use compose_plots to update all
        # visualizations at once
        fig = compose_plots([zv, wv, av, gv], 
                            epoch=0, 
                            title=f'Activation: {act} - Initializer: Normal $\sigma = {std}$')
        plt.savefig(os.path.join(output_root, f"{filename}.png"))

In [ ]:
X, y = load_data()
replaydata = ReplayData(X, y, filename='model_files/hyperparms_in_action.h5', group_name='part1')

Note that the majority of this notebook was run on Google co-lab but I needed to use Python 3.6 to get deepreplay to work so I actually ran the following plotting code on a local instance. Plots are included separately but the code below is what I ran to get them.

In [ ]:
plots_folder = 'plots'
std_list = [0.01, 0.1, 1]
norm_activations = ['tanh', 'sigmoid']

In [ ]:
for act in norm_activations:
    for std in std_list:
        filename = f'normal_initializer_{act}_{std}'
        group_name = f'normal_{act}_{std}'
        # Uses normal initializer
        init = keras.initializers.normal(mean=0, stddev=std, seed=12)
        
        plot_initializations(filename, group_name, init, act, output_root = os.getcwd())

In [3]:
for act in norm_activations:
    for std in std_list:  
        display(Image.open(os.path.join(plots_folder, f'normal_initializer_{act}_{std}.png')))

'/Users/saraprice/Documents/NYU/Spring2022/DL_Systems/Deep_Learning_Systems/weight_intialization--dead_neurons--leaky_relu'

For sigmoid with std = 0.01, we see the gradient completely vanishes and activations and Z values are in too small of a range. When we increase to std = 0.1 we still see the vanishing gradient but the activations and Z-values do seem to be in a better range. With std = 1 we no longer have a vanishing gradient but the Z-values have too wide a range and activations are in "binary mode".

For tanh with std = 0.01 we start with a vanished gradient at the last layer and Z-values and activations are in too small of ranges. For 0.1 we don't see vanishing gradient anymore and Z values and activations are in good ranges so this looks better. When we increase std too much to 1 though we see no vanishing vgradient but the binary activations and wide range of Z-values like with sigmoid.

### **Tanh and Sigmoid with Glorot Initialization**

In [ ]:
for act in norm_activations:
    filename = f'glorot_initializer_{act}'
    group_name = f'glorot_{act}'

    # Uses normal initializer
    init = keras.initializers.glorot_normal(mean=0, stddev=std, seed=12)

    plot_initializations(filename, group_name, init, act, output_root = os.getcwd())

In [ ]:
for act in norm_activations:
    display(Image.open(os.path.join(prob2_folder, f'glorot_initializer_{act}.png')))

Glorot normalization clearly helps Tanh by preventing vanishing gradients and keep activiations in a good range. However, glorot normalization does not appear to help sigmoid activation so much because the gradient fully vanishes by the first layer. 

### **ReLU activation w/ Glorot normal initialization**

In [ ]:
filename = f'glorot_initializer_relu'
group_name = f'glorot_relu'

# Uses normal initializer
init = keras.initializers.glorot_normal(seed=12)

plot_initializations(filename, group_name, init, 'relu', output_root = os.getcwd())

In [ ]:
display(Image.open(os.path.join(plots_folder, f'glorot_initializer_relu.png')))

We can see that glorot normalization does not work as well for ReLU given the gradients essentially vanish/ are on a small scale and the activations don't get very high. Also we want the Z-values to be more uniform across all the layers.


### **ReLU Activation w/ He Initialization**

In [ ]:
filename = f'he_initializer_relu'
group_name = f'he_relu'

# Uses normal initializer
init = keras.initializers.he_normal(seed=12)

plot_initializations(filename, group_name, init, 'relu', output_root = os.path.join(os.getcwd(), plots_folder))

In [ ]:
display(Image.open(os.path.join(prob2_folder, f'he_initializer_relu.png')))

He intitialization works better for ReLU because it keeps the gradients on a slighly larger scale and we see more uniformity across Z-values. Additionally the activation values are higher across the layers

## 2. 
The dying ReLU is a kind of vanishing gradient, which refers to a problem when ReLU neurons become
inactive and only output 0 for any input. In the worst case of dying ReLU, ReLU neurons at a certain
layer are all dead, i.e., the entire network dies and is referred as the dying ReLU neural networks in
Lu et al (reference below). A dying ReLU neural network collapses to a constant function. Show this
phenomenon using any one of the three 1-dimensional functions in page 13 of Lu et al. Use a 10-layer
ReLU network with width 2 (hidden units per layer). Use minibatch of 64 and draw training data
uniformly from $[\sqrt{− 7}, \sqrt{7}]$. Perform 1000 independent training simulations each with 3,000 training
points. Out of these 1000 simulations, what fraction resulted in neural network collapse. Is your answer close to over 90% as was reported in Lu et al. ? (10)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
    def __len__(self):
        return len(self.target)
    def __getitem__(self, idx):
        target = self.target[idx]
        data = self.data[idx]
        #sample = {"data": data, "label": target}
        return data, target

In [ ]:
class relu_10_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1,2)
        self.fc2 = nn.Linear(2,2)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(2, 1)
        #self.layers = nn.ModuleList([self.seq() for i in range(10)])

    def forward(self, x):
        x = self.fc1(x)
        for i in range(10):
            x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
device = torch.device("mps")
model = relu_10_layer().to(device)
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
model.train()
np.random.seed(3)
dead_network = 0
total = 0
for j in range(1000):
    train_dset = CustomDataset(training_data[:,j],targets[:,j])
    train_loader = torch.utils.data.DataLoader(train_dset, batch_size= 64)
    for i, (data, target) in enumerate(train_loader):
        data = torch.unsqueeze(torch.tensor(data), dim = 1).float().to(device)
        target = torch.tensor(target).float().to(device)
        
        output = model(data)
        optimizer.zero_grad()
        #print(torch.squeeze(output, dim =1))
        loss = criterion(torch.squeeze(output, dim =1), target)
        loss.backward()
        optimizer.step()
        if len(torch.unique(output))==1:
            dead_network+=1
        total+=1
    
    if (j%100 == 0) & (j!=0):
        print(f"After batch {j} % of dead networks: {dead_network/total*100}")


## 3. 
Instead of ReLU consider Leaky ReLU activation as defined below:
$$\begin{equation}
\phi(x) = 
\begin{cases}
  z & \text{if } z > 0 \\
  0.01z & \text{if } z \leq 0
\end{cases}
\end{equation}$$

Run the 1000 training simulations in part 2 with Leaky ReLU activation and keeping everything else same. Again calculate the fraction of simulations that resulted in neural network collapse. Did Leaky ReLU help in preventing dying neurons ?

In [ ]:
class leaky_relu_10_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1,2)
        self.fc2 = nn.Linear(2, 1)
        self.seq = nn.Sequential(nn.Linear(2,2), nn.LeakyReLU())
        #self.layers = nn.ModuleList([self.seq() for i in range(10)])

    def forward(self, x):
        x = self.fc1(x)
        for i in range(9):
            x = self.seq(x)
        x = self.fc2(x)
        return x
        

In [ ]:
device = torch.device("mps")
model = leaky_relu_10_layer().to(device)
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

model.train()
dead_network_leaky = 0
np.random.seed(4)
total = 0
for j in range(1000):
    train_dset = CustomDataset(training_data[:,j],targets[:,j])
    train_loader = torch.utils.data.DataLoader(train_dset, batch_size= 64)
    for i, (data, target) in enumerate(train_loader):
        data = torch.unsqueeze(torch.tensor(data), dim = 1).float().to(device)
        target = torch.tensor(target).float().to(device)
        output = model(data)

        optimizer.zero_grad()

        loss = criterion(torch.squeeze(output,1), target)
        loss.backward()
        optimizer.step()
        if len(torch.unique(output))==1:
            dead_network_leaky+=1
        total+=1
    
    
    if (j%100 == 0) & (j!=0):
        print(f"After batch {j} % of dead networks: {dead_network_leaky/total*100}%")





In [ ]:
print(f"% of dead networks using Leaky ReLU: {dead_network_leaky/total*100}%")

Yes leaky relu helped with the dying neurons - a far smaller percentage of simulations collapsed